In [1]:
import pandas as pd
import numpy as np
import scipy as sc

import sklearn as sk
from sklearn.model_selection import train_test_split

# Загрузка датасета

#### Получение датасета см.прошлую версию

In [2]:
sparse_dataset1 = sc.sparse.load_npz(r'D:\Bugaychenko\Netflix\sparse_dataset_super1.npz')
sparse_dataset2 = sc.sparse.load_npz(r'D:\Bugaychenko\Netflix\sparse_dataset_super2.npz')
sparse_dataset3 = sc.sparse.load_npz(r'D:\Bugaychenko\Netflix\sparse_dataset_super3.npz')
sparse_dataset4 = sc.sparse.load_npz(r'D:\Bugaychenko\Netflix\sparse_dataset_super4.npz')


# sparse_dataset = sc.sparse.vstack((sparse_dataset1, sparse_dataset2, sparse_dataset3, sparse_dataset4))

y = np.load('target_all.npy').reshape(-1, 1)

# del sparse_dataset1
# del sparse_dataset2
# del sparse_dataset3
# del sparse_dataset4


# FM

In [4]:
def lr_decrease(lr,lr_dec,epochs):
    lr = lr * np.exp(-lr_dec*epochs)
    return lr

def Nonlinear(X):
    global Z
    global X_squared
    X_squared = X.power(2)
    Z = X @ V
    #1/2 (X*V)^2 - X^2*V^2
    return 1/2 * np.sum(
        np.power(Z, 2) - X_squared @ np.power(V, 2),axis=1, keepdims=True)
        
def predict(X):
    return w0 + X @ w1 + Nonlinear(X)
    
def fit(X, Y, mini_batches=5 , batch_size = 100, lr=0.001, lr_dec=0.001, k=5):
    global w0 
    global w1 
    global V
    
    
    w0 = 0
    w1 = np.random.normal(size=(X.shape[1], 1), scale=.1)
    V = np.random.normal(size=(X.shape[1], k), scale=.1)
    n = X.shape[0]
    lr_new = lr
    
    for epoch in range(0, mini_batches):
        batch_indices = np.random.choice(n, batch_size)
        x = X[batch_indices, :]
        y = Y[batch_indices]
        y_hat = predict(x)
        dy = 2 * (y_hat - y) 
        w0 -= np.mean(dy) * lr_new 
        w1 -= x.multiply(dy).mean(axis=0).T * lr_new
        for i in range(k):
            left = x.multiply(Z[:, i].reshape(-1, 1)) #X*XV
            right = X_squared.multiply(V[:, i]) #X^2*V
            V[:, i] -= (np.asarray((left - right).multiply(dy).mean(axis=0).T) * lr_new).flatten() 
        lr_new = lr_decrease(lr,lr_dec,epoch)
        if(epoch %5000== 0):
            print(epoch)
            print(lr_new)

### Первый фолд

In [5]:
%%time
rmse_train = []
rmse_test = []


X_train = sc.sparse.vstack((sparse_dataset2, sparse_dataset3, sparse_dataset4))
y_train = y[sparse_dataset1.shape[0]:]

X_test = sparse_dataset1
y_test = y[:sparse_dataset1.shape[0]]


FM2 = fit(X_train, y_train, mini_batches=X_train.shape[0]//2500, 
          batch_size=1000, lr=0.3, lr_dec=0.00001, k=5)


0
0.3
5000
0.2853688273502142
10000
0.27145122541078787
15000
0.25821239292751735
20000
0.24561922592339452
25000
0.23364023492142144
30000
0.22224546620451535
Wall time: 1h 16min 33s


In [6]:
%%time
rmse_test.append(np.sqrt(sk.metrics.mean_squared_error(y_test, predict(X_test))))
print(rmse_test)

[1.098483049125056]
Wall time: 1min 15s


In [7]:
temp1 = sk.metrics.mean_squared_error(y_train[:y_train.shape[0]//3], predict(X_train[:X_train.shape[0]//3]))

In [8]:
temp2 = sk.metrics.mean_squared_error(y_train[y_train.shape[0]//3:2*y_train.shape[0]//3], 
                                    predict(X_train[X_train.shape[0]//3:2*X_train.shape[0]//3]))

In [9]:
temp3 = sk.metrics.mean_squared_error(y_train[2*y_train.shape[0]//3:],
                                   predict(X_train[2*X_train.shape[0]//3:]))

In [10]:
rmse_train.append(np.sqrt((temp1+temp2+temp3)/3))
print(rmse_train)

[1.0172941017615138]


### Второй фолд

In [13]:
%%time

X_train = sc.sparse.vstack((sparse_dataset1, sparse_dataset3, sparse_dataset4))
y_train = np.vstack((y[:sparse_dataset1.shape[0]],y[sparse_dataset1.shape[0] +sparse_dataset2.shape[0]:]))

X_test = sparse_dataset2
y_test = y[sparse_dataset1.shape[0]:sparse_dataset1.shape[0] +sparse_dataset2.shape[0]]

FM2 = fit(X_train, y_train, mini_batches=X_train.shape[0]//2500, 
          batch_size=1000, lr=0.3, lr_dec=0.00001, k=5)


0
0.3
5000
0.2853688273502142
10000
0.27145122541078787
15000
0.25821239292751735
20000
0.24561922592339452
25000
0.23364023492142144


In [14]:
rmse_test.append(np.sqrt(sk.metrics.mean_squared_error(y_test, predict(X_test))))
print(rmse_test)

[1.098483049125056, 1.0869790267356954]


In [15]:
temp1 = sk.metrics.mean_squared_error(y_train[:y_train.shape[0]//3], predict(X_train[:X_train.shape[0]//3]))

In [16]:
temp2 = sk.metrics.mean_squared_error(y_train[y_train.shape[0]//3:2*y_train.shape[0]//3], 
                                    predict(X_train[X_train.shape[0]//3:2*X_train.shape[0]//3]))

In [18]:
temp3 = sk.metrics.mean_squared_error(y_train[2*y_train.shape[0]//3:],
                                   predict(X_train[2*X_train.shape[0]//3:]))

In [19]:
rmse_train.append(np.sqrt((temp1+temp2+temp3)/3))

### Третий фолд

In [21]:
X_train = sc.sparse.vstack((sparse_dataset1, sparse_dataset2, sparse_dataset4))
y_train = np.vstack((y[:sparse_dataset1.shape[0]+sparse_dataset2.shape[0]],
                     y[sparse_dataset1.shape[0]+sparse_dataset2.shape[0]+sparse_dataset3.shape[0] :]
                    ))

X_test = sparse_dataset3
y_test = y[sparse_dataset1.shape[0] +sparse_dataset2.shape[0]:
           sparse_dataset1.shape[0] +sparse_dataset2.shape[0]+ sparse_dataset3.shape[0]]

FM2 = fit(X_train, y_train, mini_batches=X_train.shape[0]//2500, 
          batch_size=1000, lr=0.3, lr_dec=0.00001, k=5)


0
0.3
5000
0.2853688273502142
10000
0.27145122541078787
15000
0.25821239292751735
20000
0.24561922592339452
25000
0.23364023492142144
30000
0.22224546620451535


In [22]:
rmse_test.append(np.sqrt(sk.metrics.mean_squared_error(y_test, predict(X_test))))
print(rmse_test)

[1.098483049125056, 1.0869790267356954, 1.0954764187132677]


In [23]:
temp1 = sk.metrics.mean_squared_error(y_train[:y_train.shape[0]//3], predict(X_train[:X_train.shape[0]//3]))

In [24]:
temp2 = sk.metrics.mean_squared_error(y_train[y_train.shape[0]//3:2*y_train.shape[0]//3], 
                                    predict(X_train[X_train.shape[0]//3:2*X_train.shape[0]//3]))

In [25]:
temp3 = sk.metrics.mean_squared_error(y_train[2*y_train.shape[0]//3:],
                                   predict(X_train[2*X_train.shape[0]//3:]))

In [26]:
rmse_train.append(np.sqrt((temp1+temp2+temp3)/3))
print(rmse_train)

[1.0172941017615138, 1.018692417444757, 1.0162636024456637]


### Четвертый фолд

In [27]:
X_train = sc.sparse.vstack((sparse_dataset1, sparse_dataset2, sparse_dataset3))
y_train = y[:sparse_dataset1.shape[0]+sparse_dataset2.shape[0]+sparse_dataset3.shape[0]]

X_test = sparse_dataset4
y_test = y[sparse_dataset1.shape[0]+sparse_dataset2.shape[0]+sparse_dataset3.shape[0]:]

FM2 = fit(X_train, y_train, mini_batches=X_train.shape[0]//2500, 
          batch_size=1000, lr=0.3, lr_dec=0.00001, k=5)


0
0.3
5000
0.2853688273502142
10000
0.27145122541078787
15000
0.25821239292751735
20000
0.24561922592339452
25000
0.23364023492142144


In [28]:
rmse_test.append(np.sqrt(sk.metrics.mean_squared_error(y_test, predict(X_test))))
print(rmse_test)

[1.098483049125056, 1.0869790267356954, 1.0954764187132677, 1.0885702873164755]


In [29]:
temp1 = sk.metrics.mean_squared_error(y_train[:y_train.shape[0]//3], predict(X_train[:X_train.shape[0]//3]))

In [30]:
temp2 = sk.metrics.mean_squared_error(y_train[y_train.shape[0]//3:2*y_train.shape[0]//3], 
                                    predict(X_train[X_train.shape[0]//3:2*X_train.shape[0]//3]))

In [31]:
temp3 = sk.metrics.mean_squared_error(y_train[2*y_train.shape[0]//3:],
                                   predict(X_train[2*X_train.shape[0]//3:]))

In [32]:
rmse_train.append(np.sqrt((temp1+temp2+temp3)/3))
print(rmse_train)

[1.0172941017615138, 1.018692417444757, 1.0162636024456637, 1.0202616981833077]


In [33]:
df = pd.DataFrame(np.vstack([rmse_train, rmse_test]), index=[
    'rmse_train',
    'rmse_test',
])

df = pd.concat([df, df.mean(axis=1).rename('mean'), df.std(axis=1).rename('std')], axis=1)

df

,0,1,2,3,mean,std
rmse_train,1.017294,1.018692,1.016264,1.020262,1.018128,0.001736
rmse_test,1.098483,1.086979,1.095476,1.088570,1.092377,0.005493
